# Lezione 38 — Introduzione ai Modelli Generativi

**Obiettivi della lezione**
- Distinguere generazione vs predizione nel contesto dei modelli statistici.
- Capire LLM come modelli probabilistici di sequenze (token).
- Introdurre la probabilità condizionata e il ruolo della catena di Markov testuale.
- Riconoscere hallucinations e rischi operativi/interpretativi per un data analyst.

## Teoria concettuale approfondita

**Generazione vs predizione**
- Predizione: stimare $y$ dato $x$ (es. regressione, classificazione). Output è etichetta/valore.
- Generazione: produrre nuove istanze coerenti con la distribuzione dei dati ($p(x)$ o $p(x|c)$). Output è una sequenza/oggetto.

**LLM come modelli probabilistici**
- Vettore di token: il testo è scomposto in unità (token). Un LLM modella $p(t_i | t_1, ..., t_{i-1})$.
- Autoregressione: la sequenza è generata un token alla volta campionando dalla distribuzione condizionata precedente.

**Probabilità condizionata e catena**
- Catena: $p(t_1, ..., t_n) = \prod_{i=1}^n p(t_i | t_1, ..., t_{i-1})$.
- In pratica, i modelli moderni usano contesto ampio (finestra) e embedding per rappresentare i token.

**Hallucinations**
- Output plausibile ma non supportato dai dati di addestramento o dal prompt; deriva dal campionamento di probabilità alte ma non verificate.
- Fattori: temperature alte, prompt ambiguo, mancanza di grounding in fonti affidabili.

**Rischi operativi**
- Informazioni non verificate, bias, fuga di dati sensibili se input non filtrati.
- Interpretazione errata di probabilità: alta probabilità non implica verità fattuale.

**Perché non fare deep learning operativo qui**
- Obiettivo: capire il concetto probabilistico, non addestrare o fine-tunare reti complesse.

## Schema mentale / mappa logica
- **Quando usare modelli generativi**: sintesi di testo, summarization, assistenza alla scrittura, prototipazione di idee; generazione di esempi sintetici per test.
- **Quando NON usare**: decisioni regolamentate, report ufficiali senza verifica, dati sensibili non filtrati, domini dove l’errore costa caro.
- **Segnali pratici**: output “plausibile ma non verificato” → serve controllo umano; temperature alte amplificano creatività e rischio; prompt ambiguo aumenta la variabilità.
- **Pattern operativo**: definire il contesto, restringere il dominio, aggiungere fonti di verità (grounding), valutare l’output con checklist di qualità.

## Notebook dimostrativo (modello generativo minimale con probabilità condizionata)
Costruiamo un semplice modello bigram (catena di Markov di ordine 1) su un piccolo corpus testuale per mostrare:
- stima di $p(t_i | t_{i-1})$;
- generazione autoregressiva token-per-token;
- effetti di temperatura/rumore sulla variabilità e sulle “hallucinations”.

In [ ]:
# Importiamo librerie base per contare frequenze e generare testo
import random
from collections import Counter, defaultdict

random.seed(42)  # riproducibilità

In [ ]:
# Definiamo un piccolo corpus testuale su un dominio specifico (assistenza clienti)
corpus = [
    "spedizione standard richiede tre giorni lavorativi",
    "spedizione express richiede un giorno lavorativo",
    "resi accettati entro trenta giorni con ricevuta",
    "rimborsi gestiti dal reparto finance con approvazione",
]
corpus

In [ ]:
# Tokenizziamo in modo minimale e costruiamo frequenze bigram

def tokenize(sentence):
    return sentence.lower().split()

bigrams = defaultdict(Counter)
for sent in corpus:
    tokens = ["<s>"] + tokenize(sent) + ["</s>"]  # aggiungiamo start/end
    for w1, w2 in zip(tokens, tokens[1:]):
        bigrams[w1][w2] += 1

# Calcoliamo distribuzioni condizionate p(w2|w1)
bigram_probs = {}
for w1, counter in bigrams.items():
    total = sum(counter.values())
    bigram_probs[w1] = {w2: cnt / total for w2, cnt in counter.items()}

# Mostriamo le probabilità di alcune transizioni
bigram_probs["spedizione"], bigram_probs["richiede"]

In [ ]:
# Funzione di generazione autoregressiva con temperatura

def sample_next(probs: dict, temperature: float = 1.0):
    # Applichiamo temperatura ai log-prob per controllare la randomicità
    items = list(probs.items())
    tokens, p = zip(*items)
    if temperature != 1.0:
        import math

        logits = [math.log(pi + 1e-12) / temperature for pi in p]
        # Normalizziamo per ottenere una distribuzione valida
        max_logit = max(logits)
        exp_logits = [math.exp(l - max_logit) for l in logits]
        total = sum(exp_logits)
        p = [v / total for v in exp_logits]
    return random.choices(tokens, weights=p, k=1)[0]


def generate(max_len=15, temperature=1.0):
    token = "<s>"
    output = []
    for _ in range(max_len):
        probs = bigram_probs.get(token, None)
        if not probs:
            break
        nxt = sample_next(probs, temperature=temperature)
        if nxt == "</s>":
            break
        output.append(nxt)
        token = nxt
    return " ".join(output)

# Generiamo con due temperature per confrontare coerenza vs variabilità
gen_cool = generate(temperature=0.7)
gen_hot = generate(temperature=1.5)
(gen_cool, gen_hot)

### Osservazioni sul risultato
- Temperatura più bassa (0.7) privilegia i token più probabili ⇒ frasi più prevedibili e coerenti.
- Temperatura alta (1.5) aumenta l’esplorazione ⇒ più variabilità, maggiore rischio di incoerenza (hallucination in scala toy).
- Il modello bigram ignora contesto lungo: può produrre frasi grammaticali ma semantica limitata. LLM usano contesti più lunghi e rappresentazioni dense.

## Esercizi svolti (step-by-step)
Esercizi per consolidare: stima di probabilità condizionate, effetto della temperatura, gestione di contesto limitato.

In [ ]:
# Esercizio 1: calcolare la probabilità di una frase corta secondo il modello bigram

sentence = "spedizione standard richiede".split()
log_prob = 0.0
for w1, w2 in zip(["<s>"] + sentence, sentence + ["</s>"]):
    p = bigram_probs.get(w1, {}).get(w2, 1e-12)  # smoothing minimale per zero-freq
    import math

    log_prob += math.log(p)
log_prob

In [ ]:
# Esercizio 2: osservare l'effetto della temperatura su più campioni
for temp in [0.6, 1.0, 1.4]:
    samples = [generate(temperature=temp) for _ in range(3)]
    print(f"T={temp}: {samples}")

In [ ]:
# Esercizio 3: aggiungere un nuovo documento per ridurre hallucinations nel dominio
nuovo_doc = "spedizione express spesso richiede approvazione speciale"
corpus_esteso = corpus + [nuovo_doc]

# Ricostruiamo bigrammi con il nuovo documento
bigrams_ext = defaultdict(Counter)
for sent in corpus_esteso:
    tokens = ["<s>"] + tokenize(sent) + ["</s>"]
    for w1, w2 in zip(tokens, tokens[1:]):
        bigrams_ext[w1][w2] += 1

bigram_probs_ext = {}
for w1, counter in bigrams_ext.items():
    total = sum(counter.values())
    bigram_probs_ext[w1] = {w2: cnt / total for w2, cnt in counter.items()}

# Generiamo dal modello esteso per vedere se compaiono termini più coerenti con express
def generate_ext(max_len=15, temperature=1.0):
    token = "<s>"
    output = []
    for _ in range(max_len):
        probs = bigram_probs_ext.get(token, None)
        if not probs:
            break
        nxt = sample_next(probs, temperature=temperature)
        if nxt == "</s>":
            break
        output.append(nxt)
        token = nxt
    return " ".join(output)

[generate_ext(temperature=0.9) for _ in range(3)]

## Conclusione operativa
- Portarsi a casa: un modello generativo produce sequenze campionando da distribuzioni condizionate; la temperatura controlla creatività vs coerenza.
- Errori da evitare: confondere alta probabilità con verità; usare output senza verifica; non limitare il dominio o aggiungere grounding.
- Ponte verso la prossima lezione: AI nel mondo reale — decision support, automazione e ruolo umano nel controllo di qualità.